Задачи:

1) Скачать датасеты airports.dat и routes.dat с сайта https://openflights.org/data.html

2) Осуществить выгрузку данных как DataFrame, при этом добавить правильные названия колонок к данным.

3) Добавить столбец в DataFrame с названием континента на основе airports.dat.

4) Случайным образом выбрать континент (кроме Антарктиды, код должен работать для любого континента).

5) Для отобранного континента, для каждой страны данного континента отобрать три наиболее загруженных аэропорта по количеству взлет/посадок (примечание – только для аэропортов с кодом IATA). Взлет/посадки нужно посчитать, используя код IATA в базе routes.dat. Результат оформить в виде Series: индекс 0 уровня – страна, индекс 1 уровня -полное название аэропорта, значение – количество взлет/посадок, упорядочить страны по алфавиту, аэропорты внутри страны от наибольшего к наименьшему.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import awoc

In [2]:
url_airport = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
url_routers = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'

In [3]:
columns_airport = ['Airport ID', 'Name', 'City',
                   'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
                   'DST', 'Tz database time zone', 'Type', 'Source']

columns_routers = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 
                   'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

df_raw_airport = pd.read_csv(url_airport, names = columns_airport).replace('\\N', np.nan)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers, names = columns_routers).replace('\\N', np.nan)
display(df_raw_routers.head(3))

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.82679,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports


,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


In [4]:
world = awoc.AWOC()

def country_to_continent(country):
    try:
        continent = world.get_country_continent_name(country)
    except NameError:
        continent = np.nan
    return continent

In [5]:
dict_nan_country = {"Cote d'Ivoire" : 'Ivory Coast',
                    'Congo (Brazzaville)' : 'Republic of the Congo',
                    'Congo (Kinshasa)' : 'Democratic Republic of the Congo',
                    'West Bank' : 'Palestine',
                    'Burma' : 'Myanmar',
                    }
df_raw_airport['Country'] = df_raw_airport['Country'].replace(dict_nan_country)
df_raw_airport['Continent'] = df_raw_airport['Country'].map(country_to_continent)

dict_unknown_country = {'Midway Islands' : 'Oceania',
                        'French Guiana' : 'South America',
                        'Martinique' : 'North America',
                        'Guadeloupe' : 'North America',
                        'Virgin Islands' : 'North America',
                        'Norfolk Island' : 'Oceania', 
                        'Johnston Atoll' : 'Oceania',
                        'Cocos (Keeling) Islands' : 'Asia',
                        'Svalbard' : 'Europe',
                        'Wake Island' : 'Oceania'
                       }
df_raw_airport['Continent'] = df_raw_airport['Continent'].fillna(df_raw_airport["Country"].map(dict_unknown_country))

In [6]:
all_continents = df_raw_airport['Continent'].tolist()
all_continents_unique = list(set(all_continents))
all_continents_unique.remove('Antarctica')
random_continent = list(np.random.choice(all_continents_unique, 1))
random_continent

['Europe']

In [7]:
table_of_random_continent = df_raw_airport.loc[(df_raw_airport.Continent == random_continent[0])]
table_of_random_continent_with_IATA = table_of_random_continent.loc[~table_of_random_continent['IATA'].isin([np.nan])]
table_of_random_continent_with_IATA

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent
10,11,Akureyri Airport,Akureyri,Iceland,AEY,BIAR,65.660004,-18.072701,6,0,N,Atlantic/Reykjavik,airport,OurAirports,Europe
11,12,Egilsstaðir Airport,Egilsstadir,Iceland,EGS,BIEG,65.283302,-14.401400,76,0,N,Atlantic/Reykjavik,airport,OurAirports,Europe
12,13,Hornafjörður Airport,Hofn,Iceland,HFN,BIHN,64.295601,-15.227200,24,0,N,Atlantic/Reykjavik,airport,OurAirports,Europe
13,14,Húsavík Airport,Husavik,Iceland,HZK,BIHU,65.952301,-17.426001,48,0,N,Atlantic/Reykjavik,airport,OurAirports,Europe
14,15,Ísafjörður Airport,Isafjordur,Iceland,IFJ,BIIS,66.058098,-23.135300,8,0,N,Atlantic/Reykjavik,airport,OurAirports,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7502,13432,Talakan Airport,Talakan,Russia,TLK,UECT,59.876389,111.044444,1329,9,U,NaN,airport,OurAirports,Europe
7622,13677,Algeciras Heliport,Algeciras,Spain,AEI,LEAG,36.128880,-5.441118,98,-2,E,NaN,airport,OurAirports,Europe
7669,13829,Región de Murcia International Airport,Murcia,Spain,RMU,LEMI,37.803000,-1.125000,632,1,E,NaN,airport,OurAirports,Europe
7676,14088,Bezmer Air Base,Yambol,Bulgaria,JAM,LBIA,42.454899,26.352200,509,NaN,NaN,NaN,airport,OurAirports,Europe


In [8]:
def sum_of_flying(IATA_code):
    summa = len(df_raw_routers.loc[(df_raw_routers['Source airport'] == IATA_code)| (df_raw_routers['Destination airport'] == IATA_code)])
    return summa

In [9]:
table_of_random_continent_with_IATA = table_of_random_continent_with_IATA.copy()

In [10]:
table_of_random_continent_with_IATA['Sum'] = table_of_random_continent_with_IATA['IATA'].map(sum_of_flying)
table_of_random_continent_with_IATA_cut = table_of_random_continent_with_IATA[['Country', 'Name', 'Sum']]
busiest_airports = table_of_random_continent_with_IATA_cut.groupby("Country").apply(lambda x: x.nlargest(3, "Sum")).set_index(['Country', 'Name'])
busiest_airports

Sum
Country        Name                                            
Albania        Tirana International Airport Mother Teresa    72
Austria        Vienna International Airport                 616
               Salzburg Airport                              69
               Graz Airport                                  38
Belarus        Minsk National Airport                       108
...                                                         ...
Ukraine        Kiev Zhuliany International Airport           44
               Odessa International Airport                  37
United Kingdom London Heathrow Airport                     1051
               London Gatwick Airport                       708
               Manchester Airport                           627

[118 rows x 1 columns]

In [11]:
three_largest_countries = busiest_airports.groupby('Country')['Sum'].sum().nlargest(3)
three_country = three_largest_countries.index.tolist()

In [12]:
three_busiest_airports = busiest_airports.loc[three_country]
three_busiest_airports = three_busiest_airports.sort_values(by=['Country', 'Sum'], ascending=[True, False])
three_busiest_airports_reset_index = three_busiest_airports.reset_index()

In [13]:
three_busiest_airports_SERIES = pd.Series(three_busiest_airports_reset_index['Sum'].values, 
                        index = [three_busiest_airports_reset_index['Country'], three_busiest_airports_reset_index['Name']])
three_busiest_airports_SERIES.sort_index()
display(three_busiest_airports_SERIES)
print('TYPE:', type(three_busiest_airports_SERIES))

Country         Name                                
Germany         Frankfurt am Main Airport                990
                Munich Airport                           728
                Düsseldorf Airport                       570
Spain           Barcelona International Airport          783
                Adolfo Suárez Madrid–Barajas Airport     661
                Palma De Mallorca Airport                548
United Kingdom  London Heathrow Airport                 1051
                London Gatwick Airport                   708
                Manchester Airport                       627
dtype: int64

TYPE: <class 'pandas.core.series.Series'>
